This notebook covers the modelling techniques used for getting optimal prediction results.

## Data Cleaning and Preparation

This involves the necessary steps and processing to get both the train and test datasets in an acceptable form for the machine learnning algorithm:

* Creating validation data sets
* Selecting interested logs
* Resolving missing values
* Encoding categorical variables
* Data augmentation

In [65]:
import numpy as np
import random
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score

In [2]:
traindata = pd.read_csv('./data/train.csv', sep=';')
testdata = pd.read_csv('./data/leaderboard_test_features.csv.txt', sep=';')

In [3]:
A = np.load('./data/penalty_matrix.npy')   # penalty matrix used for scoring

In [5]:
A

array([[0.   , 2.   , 3.5  , 3.   , 3.75 , 3.5  , 3.5  , 4.   , 4.   ,
        2.5  , 3.875, 3.25 ],
       [2.   , 0.   , 2.375, 2.75 , 4.   , 3.75 , 3.75 , 3.875, 4.   ,
        3.   , 3.75 , 3.   ],
       [3.5  , 2.375, 0.   , 2.   , 3.5  , 3.5  , 3.75 , 4.   , 4.   ,
        2.75 , 3.25 , 3.   ],
       [3.   , 2.75 , 2.   , 0.   , 2.5  , 2.   , 2.25 , 4.   , 4.   ,
        3.375, 3.75 , 3.25 ],
       [3.75 , 4.   , 3.5  , 2.5  , 0.   , 2.625, 2.875, 3.75 , 3.25 ,
        3.   , 4.   , 3.625],
       [3.5  , 3.75 , 3.5  , 2.   , 2.625, 0.   , 1.375, 4.   , 3.75 ,
        3.5  , 4.   , 3.625],
       [3.5  , 3.75 , 3.75 , 2.25 , 2.875, 1.375, 0.   , 4.   , 3.75 ,
        3.125, 4.   , 3.75 ],
       [4.   , 3.875, 4.   , 4.   , 3.75 , 4.   , 4.   , 0.   , 2.75 ,
        3.75 , 3.75 , 4.   ],
       [4.   , 4.   , 4.   , 4.   , 3.25 , 3.75 , 3.75 , 2.75 , 0.   ,
        4.   , 4.   , 3.875],
       [2.5  , 3.   , 2.75 , 3.375, 3.   , 3.5  , 3.125, 3.75 , 4.   ,
        0.   , 2.5  

Scoring matrix for petrophysical interpretation

![Penalty.png](images/penalty_matrix.jpg)

In [6]:
def score(y_true, y_pred):

    '''
    custom metric used for evaluation
    args:
      y_true: actual prediction
      y_pred: predictions made
    '''

    S = 0.0
    y_true = y_true.astype(int)
    y_pred = y_pred.astype(int)
    for i in range(0, y_true.shape[0]):
        S -= A[y_true[i], y_pred[i]]
    return S/y_true.shape[0]

### Creating Validation Sets

Validation sets are created to properly evaluate changes made on the machine learning model. This is important to prevent overfitting the open test data since the blind test is used as the final determiner. So it is important to build ML models that will generalise better on unseen wells. Having no idea how the blind wells would come (geospatial distribution, logs presence), there was no specific guide in selecting train wells, so wells were randomly selected from the train to create two validation sets (each comprising of 10 wells). 

In [27]:
train_wells = traindata.WELL.unique()

In [28]:
print(f'Initial total number of train wells: {len(train_wells)}')

Initial total number of train wells: 98


In [43]:
np.random.seed(12)

In [44]:
valid1 = random.sample(list(train_wells), 10)   #randomly sample 10 well names from train data

In [45]:
valid1

['31/6-5',
 '34/7-13',
 '30/3-5 S',
 '34/10-19',
 '34/11-2 S',
 '15/9-15',
 '35/3-7 S',
 '31/4-5',
 '35/11-15 S',
 '31/3-4']

QC to remove valid1 wells from train wells to prevent having same well(s) in the second validation data

In [46]:
train_wells = [well for well in train_wells if not well in valid1]
print(f'Number of wells left: {len(train_wells)}')

Number of wells left: 68


In [47]:
valid2 = random.sample(list(train_wells), 10)

train_wells = [well for well in train_wells if not well in valid2]
print(f'Number of wells left: {len(train_wells)}')

Number of wells left: 58


In [48]:
print(len(valid1), len(valid2))

10 10


In [49]:
validation_wells = set(valid1 + valid2)
print(len(validation_wells))

20


Let's proceed to getting the validation data from the train data set and dropping them to prevent any form of data leakage.

In [50]:
def create_validation_set(train, wells):
    
    '''
    Function to validation sets from the full train data using well names
    '''
    
    validation = pd.DataFrame(columns=list(train.columns))
    
    for well in wells:
        welldata = train.loc[train.WELL == well]
        validation = pd.concat((welldata, validation))
        
    return validation

In [51]:
# using function to get data for validation wells

validation1 = create_validation_set(traindata, valid1)
validation2 = create_validation_set(traindata, valid2)

In [52]:
validation2

,WELL,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,GROUP,FORMATION,CALI,RSHA,RMED,...,ROP,DTS,DCAL,DRHO,MUDWEIGHT,RMIC,ROPA,RXO,FORCE_2020_LITHOFACIES_LITHOLOGY,FORCE_2020_LITHOFACIES_CONFIDENCE
924195,34/8-1,891.137023,469664.71875,6803719.0,-868.119202,NORDLAND GP.,NaN,NaN,NaN,2.342517,...,NaN,NaN,NaN,-0.722545,NaN,NaN,NaN,NaN,65000,2.0
924196,34/8-1,891.289023,469664.71875,6803719.0,-868.271179,NORDLAND GP.,NaN,NaN,NaN,2.310889,...,NaN,NaN,NaN,-0.698928,NaN,NaN,NaN,NaN,65000,2.0
924197,34/8-1,891.441023,469664.71875,6803719.0,-868.423218,NORDLAND GP.,NaN,NaN,NaN,2.320935,...,NaN,NaN,NaN,-0.684372,NaN,NaN,NaN,NaN,65000,2.0
924198,34/8-1,891.593023,469664.71875,6803719.0,-868.575195,NORDLAND GP.,NaN,NaN,NaN,2.275842,...,NaN,NaN,NaN,-0.661161,NaN,NaN,NaN,NaN,65000,2.0
924199,34/8-1,891.745023,469664.71875,6803719.0,-868.727173,NORDLAND GP.,NaN,NaN,NaN,2.223282,...,NaN,NaN,NaN,-0.662985,NaN,NaN,NaN,NaN,65000,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1152225,36/7-3,2947.576000,556082.31250,6810860.0,-2922.401367,VIKING GP.,Heather Fm.,NaN,NaN,21.895206,...,NaN,NaN,NaN,NaN,NaN,NaN,9.90472,NaN,65030,2.0
1152226,36/7-3,2947.728000,556082.31250,6810860.0,-2922.553467,VIKING GP.,Heather Fm.,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,9.63416,NaN,65030,2.0
1152227,36/7-3,2947.880000,NaN,NaN,NaN,VIKING GP.,Heather Fm.,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,9.36360,NaN,65030,2.0
1152228,36/7-3,2948.032000,NaN,NaN,NaN,VIKING GP.,Heather Fm.,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65030,2.0


In [53]:
# total validation data

validation = pd.concat((validation1, validation2))

In [54]:
validation.shape, validation1.shape, validation2.shape

((232134, 29), (99463, 29), (132671, 29))

In [55]:
# dropping validation data from train data

new_train = pd.concat([traindata, validation, validation]).drop_duplicates(keep=False)
print(f'Previous train data shape: {traindata.shape}')
print(f'New train data shape: {new_train.shape}')

Previous train data shape: (1170511, 29)
New train data shape: (938377, 29)


In [56]:
# QC to ensure there are no data leakage

previous_rows = traindata.shape[0]
new_train_rows = new_train.shape[0]
validation_rows = validation.shape[0]

print(f'Number of previous train data rows: {previous_rows}')
print(f'Validation + new train rows: {validation_rows+new_train_rows}')

Number of previous train data rows: 1170511
Validation + new train rows: 1170511


In [57]:
# to confirm we still have all samples of the labels in the train data set

new_train.FORCE_2020_LITHOFACIES_LITHOLOGY.value_counts()

65000    577878
30000    130927
65030    123298
70000     44499
80000     27737
99000     12229
70032      8664
88000      8213
90000      2527
74000      1292
86000      1010
93000       103
Name: FORCE_2020_LITHOFACIES_LITHOLOGY, dtype: int64

Now we can proceed to other preparation procedures.

Logs were selected based on user's desire to use them for training. The confidence logs was dropped as this was absent in the test logs as the ML models need the same set of wells used for training in making predictions on the test wells. Other absent logs and logs with low percentage of values from the combined test data are also dropped. This has previously been visualized in the EDA notebook. A cut off of 30% may be selected as criteria for dropping the logs. Let's take a look at that!

In [58]:
print(f'Percentage of values in test logs:')
100 - testdata.isna().sum()/testdata.shape[0]*100

Percentage of values in test logs:


WELL         100.000000
DEPTH_MD     100.000000
X_LOC         99.956867
Y_LOC         99.956867
Z_LOC         99.956867
GROUP        100.000000
FORMATION     94.828418
CALI          95.873116
RSHA          28.582603
RMED          99.570863
RDEP          99.956867
RHOB          87.601070
GR           100.000000
SGR            0.000000
NPHI          76.062609
PEF           82.978521
DTC           99.398330
SP            48.708932
BS            48.955302
ROP           49.943708
DTS           31.596801
DCAL           9.880397
DRHO          81.555130
MUDWEIGHT     14.818037
RMIC           8.272776
ROPA          40.786338
RXO           21.820947
dtype: float64

For better and faster processing, the train, validation and test data sets will be concatenated and processed together as we need these data sets to be in the same formats to get good predictions out of the ML model. But let's have it in mind that the RSHA, SGR, DCAL, MUDWEIGHT, RMIC and RXO will be dropped from the wells. Let's proceed.

Let's extract the data sets indices that will be used for splitting the features and targets into their respective datasets after prepration is complete. We will also be extracting the train target

In [59]:
ntrain = new_train.shape[0]
ntest = testdata.shape[0]
nvalid1 = validation1.shape[0]
nvalid2 = validation2.shape[0]
nvalid3 = validation.shape[0]

df = pd.concat((new_train, testdata, validation1, validation2, validation)).reset_index(drop=True)

So the combined dataframe for preparation

![Picture.png](images/Picture3.png)

In [60]:
df.shape

(1539431, 29)

The procedure below is used to extract data needed for the augmentation procedure to be performed after every other preparation has been done

In [61]:
#making a copy of the dataframes

train = new_train.copy()
test = testdata.copy()
valid1 = validation1.copy()
valid2 = validation2.copy()
valid = validation.copy()

In [62]:
# extracting the data sets well names and depth values needed for augmentation

train_well = train.WELL.values
train_depth = train.DEPTH_MD.values

test_well = test.WELL.values
test_depth = test.DEPTH_MD.values
 
valid1_well = valid1.WELL.values
valid1_depth = valid1.DEPTH_MD.values
 
valid2_well = valid2.WELL.values
valid2_depth = valid2.DEPTH_MD.values
 
valid_well = valid.WELL.values
valid_depth = valid.DEPTH_MD.values

Now let's extract the data sets labels and prepare them for training and validation performance check.

In [63]:
lithology = train['FORCE_2020_LITHOFACIES_LITHOLOGY']
valid1_lithology = valid1['FORCE_2020_LITHOFACIES_LITHOLOGY']
valid2_lithology = valid2['FORCE_2020_LITHOFACIES_LITHOLOGY']
valid_lithology = valid['FORCE_2020_LITHOFACIES_LITHOLOGY']
 
lithology_numbers = {30000: 0,
                 65030: 1,
                 65000: 2,
                 80000: 3,
                 74000: 4,
                 70000: 5,
                 70032: 6,
                 88000: 7,
                 86000: 8,
                 99000: 9,
                 90000: 10,
                 93000: 11}
 
lithology = lithology.map(lithology_numbers)
valid1_lithology = valid1_lithology.map(lithology_numbers)
valid2_lithology = valid2_lithology.map(lithology_numbers)
valid_lithology = valid_lithology.map(lithology_numbers)

In [64]:
valid1_lithology

635461    2
635462    2
635463    2
635464    2
635465    2
         ..
685276    2
685277    2
685278    2
685279    2
685280    2
Name: FORCE_2020_LITHOFACIES_LITHOLOGY, Length: 99463, dtype: int64

### Data Encoding

The categorical logs/columns in the data set need to be encoded for use by the ML algorithm. From the data visualization, we saw the high dimensionality of the logs (especially the FORMATION log with 69 distinct values), so label encoding will be applied instead of one hot encoding these features to prevent high dimensionality of the data.

In [66]:
df.shape

(1539431, 29)

In [67]:
df['GROUP_encoded'] = df['GROUP'].astype('category')
df['GROUP_encoded'] = df['GROUP_encoded'].cat.codes 

df['FORMATION_encoded'] = df['FORMATION'].astype('category')
df['FORMATION_encoded'] = df['FORMATION_encoded'].cat.codes

df['WELL_encoded'] = df['WELL'].astype('category')
df['WELL_encoded'] = df['WELL_encoded'].cat.codes

print(f'shape of dataframe after label encoding columns {df.shape}')

shape of dataframe after label encoding columns (1539431, 32)


In [68]:
df.head()

,WELL,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,GROUP,FORMATION,CALI,RSHA,RMED,...,DRHO,MUDWEIGHT,RMIC,ROPA,RXO,FORCE_2020_LITHOFACIES_LITHOLOGY,FORCE_2020_LITHOFACIES_CONFIDENCE,GROUP_encoded,FORMATION_encoded,WELL_encoded
0,15/9-13,494.528,437641.96875,6470972.5,-469.501831,NORDLAND GP.,NaN,19.480835,NaN,1.611410,...,-0.574928,NaN,NaN,NaN,NaN,65000,1.0,6,-1,0
1,15/9-13,494.680,437641.96875,6470972.5,-469.653809,NORDLAND GP.,NaN,19.468800,NaN,1.618070,...,-0.570188,NaN,NaN,NaN,NaN,65000,1.0,6,-1,0
2,15/9-13,494.832,437641.96875,6470972.5,-469.805786,NORDLAND GP.,NaN,19.468800,NaN,1.626459,...,-0.574245,NaN,NaN,NaN,NaN,65000,1.0,6,-1,0
3,15/9-13,494.984,437641.96875,6470972.5,-469.957794,NORDLAND GP.,NaN,19.459282,NaN,1.621594,...,-0.586315,NaN,NaN,NaN,NaN,65000,1.0,6,-1,0
4,15/9-13,495.136,437641.96875,6470972.5,-470.109772,NORDLAND GP.,NaN,19.453100,NaN,1.602679,...,-0.597914,NaN,NaN,NaN,NaN,65000,1.0,6,-1,0


In [69]:
df.GROUP_encoded.value_counts()

 5     375804
 9     325375
 3     162759
 7     162020
 12    157759
 6     152890
 2      75038
 0      51283
 11     33374
 4      21425
 13     13021
 1       3125
 8       3075
-1       1278
 10      1205
Name: GROUP_encoded, dtype: int64

In [70]:
# dropping the previous columns after encoding

df = df.drop(['WELL', 'GROUP', 'FORMATION'], axis=1)

In [71]:
df.isna().sum()/df.shape[0]*100

DEPTH_MD                              0.000000
X_LOC                                 0.703961
Y_LOC                                 0.703961
Z_LOC                                 0.703961
CALI                                  7.569550
RSHA                                 48.778672
RMED                                  2.609081
RDEP                                  0.719552
RHOB                                 13.044755
GR                                    0.000000
SGR                                  94.927606
NPHI                                 33.245335
PEF                                  39.276200
DTC                                   5.560626
SP                                   27.844509
BS                                   42.046899
ROP                                  55.059239
DTS                                  84.366496
DCAL                                 76.267595
DRHO                                 15.144362
MUDWEIGHT                            74.034887
RMIC         

### Filling Missing Values

Some fractions of missing values still exist in present logs, how do we resolve that? While we can use a mean of values in a window to solve this, backward or forward fill, we could also decide to fill up all missing values with a distinct value different from other values. This way, the ML algorithm used (in this case a gradient tree algorithm) can differentiate this better. From validation, this improved result better. -9999 is used, and since this is a classification problem as opposed to a regression where we predict actual lithology values, outlier effect is not observed in the output.

In [72]:
df = df.fillna(-9999)

In [73]:
df.isna().sum()/df.shape[0]*100

DEPTH_MD                             0.0
X_LOC                                0.0
Y_LOC                                0.0
Z_LOC                                0.0
CALI                                 0.0
RSHA                                 0.0
RMED                                 0.0
RDEP                                 0.0
RHOB                                 0.0
GR                                   0.0
SGR                                  0.0
NPHI                                 0.0
PEF                                  0.0
DTC                                  0.0
SP                                   0.0
BS                                   0.0
ROP                                  0.0
DTS                                  0.0
DCAL                                 0.0
DRHO                                 0.0
MUDWEIGHT                            0.0
RMIC                                 0.0
ROPA                                 0.0
RXO                                  0.0
FORCE_2020_LITHO

Now that we've completed the majority of the preparation, let's split back our concatenated dataframe into their validation sets, train and test set

In [74]:
data = df.copy()   #making a copy of the preparaed dataframe to work with
data.shape

(1539431, 29)

Remember the shape of the concatenated dataframe;

![Picture.png](images/Picture3.png)

using the data sets indices will be used for slicing out their corresponding features

In [75]:
train = data[:ntrain].copy()
train.drop(['FORCE_2020_LITHOFACIES_LITHOLOGY'], axis=1, inplace=True)
        
test = data[ntrain:(ntest+ntrain)].copy()
test.drop(['FORCE_2020_LITHOFACIES_LITHOLOGY'], axis=1, inplace=True)
test = test.reset_index(drop=True)

valid1 = data[(ntest+ntrain):(ntest+ntrain+nvalid1)].copy()
valid1.drop(['FORCE_2020_LITHOFACIES_LITHOLOGY'], axis=1, inplace=True)
valid1 = valid1.reset_index(drop=True)

valid2 = data[(ntest+ntrain+nvalid1):(ntest+ntrain+nvalid1+nvalid2)].copy()
valid2.drop(['FORCE_2020_LITHOFACIES_LITHOLOGY'], axis=1, inplace=True)
valid2 = valid2.reset_index(drop=True)

valid = data[(ntest+ntrain+nvalid1+nvalid2):].copy()
valid.drop(['FORCE_2020_LITHOFACIES_LITHOLOGY'], axis=1, inplace=True)
valid = valid.reset_index(drop=True)

In [76]:
# checking shapes of sliced data sets for QC

train.shape, test.shape, valid1.shape, valid2.shape, valid.shape

((938377, 28), (136786, 28), (99463, 28), (132671, 28), (232134, 28))

### Data Augmentation

The data augmentation technique is extracted from the ISPL team code for the 2016 SEG ML competition : https://github.com/seg/2016-ml-contest/tree/master/ispl . The technique was based on the assumption that "facies do not abrutly change from a given depth layer to the next one". This was implemented by aggregating features at neighbouring depths and computing the feature spatial gradient.

In [77]:
# Feature windows concatenation function
def augment_features_window(X, N_neig):
    
    # Parameters
    N_row = X.shape[0]
    N_feat = X.shape[1]
 
    # Zero padding
    X = np.vstack((np.zeros((N_neig, N_feat)), X, (np.zeros((N_neig, N_feat)))))
 
    # Loop over windows
    X_aug = np.zeros((N_row, N_feat*(2*N_neig+1)))
    for r in np.arange(N_row)+N_neig:
        this_row = []
        for c in np.arange(-N_neig,N_neig+1):
            this_row = np.hstack((this_row, X[r+c]))
        X_aug[r-N_neig] = this_row
 
    return X_aug
 
# Feature gradient computation function
def augment_features_gradient(X, depth):
    
    # Compute features gradient
    d_diff = np.diff(depth).reshape((-1, 1))
    d_diff[d_diff==0] = 0.001
    X_diff = np.diff(X, axis=0)
    X_grad = X_diff / d_diff
        
    # Compensate for last missing value
    X_grad = np.concatenate((X_grad, np.zeros((1, X_grad.shape[1]))))
    
    return X_grad
 
# Feature augmentation function
def augment_features(X, well, depth, N_neig=1):
    
    # Augment features
    X_aug = np.zeros((X.shape[0], X.shape[1]*(N_neig*2+2)))
    for w in np.unique(well):
        w_idx = np.where(well == w)[0]
        X_aug_win = augment_features_window(X[w_idx, :], N_neig)
        X_aug_grad = augment_features_gradient(X[w_idx, :], depth[w_idx])
        X_aug[w_idx, :] = np.concatenate((X_aug_win, X_aug_grad), axis=1)
    
    return X_aug

In [78]:
print(f'Shape of datasets before augmentation {train.shape, test.shape, valid1.shape, valid2.shape, valid.shape}')

aug_train = augment_features(train.values, train_well, train_depth)
aug_test = augment_features(test.values, test_well, test_depth)
aug_valid1 = augment_features(valid1.values, valid1_well, valid1_depth)
aug_valid2 = augment_features(valid2.values, valid2_well, valid2_depth)
aug_valid = augment_features(valid.values, valid_well, valid_depth)

print(f'Shape of datasets after augmentation {aug_train.shape, aug_test.shape, aug_valid1.shape, aug_valid2.shape, aug_valid.shape}')

Shape of datasets before augmentation ((938377, 28), (136786, 28), (99463, 28), (132671, 28), (232134, 28))
Shape of datasets after augmentation ((938377, 112), (136786, 112), (99463, 112), (132671, 112), (232134, 112))


In [79]:
pd.DataFrame(aug_train).head(10)

,0,1,2,3,4,5,6,7,8,9,...,102,103,104,105,106,107,108,109,110,111
0,0.000,0.00000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.031179,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,494.528,437641.96875,6470972.5,-469.501831,19.480835,-9999.0,1.611410,1.798681,1.884186,80.200851,...,0.0,-0.026689,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,494.680,437641.96875,6470972.5,-469.653809,19.468800,-9999.0,1.618070,1.795641,1.889794,79.262886,...,0.0,-0.079409,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,494.832,437641.96875,6470972.5,-469.805786,19.468800,-9999.0,1.626459,1.800733,1.896523,74.821999,...,0.0,-0.076305,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,494.984,437641.96875,6470972.5,-469.957794,19.459282,-9999.0,1.621594,1.801517,1.891913,72.878922,...,0.0,-0.024252,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,495.136,437641.96875,6470972.5,-470.109772,19.453100,-9999.0,1.602679,1.795299,1.880034,71.729141,...,0.0,0.021260,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,495.288,437641.96875,6470972.5,-470.261780,19.453100,-9999.0,1.585567,1.804719,1.879687,72.014420,...,0.0,-0.024150,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,495.440,437641.96875,6470972.5,-470.413788,19.462496,-9999.0,1.576569,1.805498,1.878731,72.588089,...,0.0,-0.081083,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,495.592,437641.96875,6470972.5,-470.565796,19.468800,-9999.0,1.587011,1.808367,1.867837,71.283051,...,0.0,-0.049004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,495.744,437641.96875,6470972.5,-470.717773,19.468800,-9999.0,1.613674,1.815813,1.847233,69.721436,...,0.0,0.065673,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [80]:
train.head(10)

,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,CALI,RSHA,RMED,RDEP,RHOB,GR,...,DCAL,DRHO,MUDWEIGHT,RMIC,ROPA,RXO,FORCE_2020_LITHOFACIES_CONFIDENCE,GROUP_encoded,FORMATION_encoded,WELL_encoded
0,494.528,437641.96875,6470972.5,-469.501831,19.480835,-9999.0,1.611410,1.798681,1.884186,80.200851,...,-9999.0,-0.574928,-9999.0,-9999.0,-9999.0,-9999.0,1.0,6,-1,0
1,494.680,437641.96875,6470972.5,-469.653809,19.468800,-9999.0,1.618070,1.795641,1.889794,79.262886,...,-9999.0,-0.570188,-9999.0,-9999.0,-9999.0,-9999.0,1.0,6,-1,0
2,494.832,437641.96875,6470972.5,-469.805786,19.468800,-9999.0,1.626459,1.800733,1.896523,74.821999,...,-9999.0,-0.574245,-9999.0,-9999.0,-9999.0,-9999.0,1.0,6,-1,0
3,494.984,437641.96875,6470972.5,-469.957794,19.459282,-9999.0,1.621594,1.801517,1.891913,72.878922,...,-9999.0,-0.586315,-9999.0,-9999.0,-9999.0,-9999.0,1.0,6,-1,0
4,495.136,437641.96875,6470972.5,-470.109772,19.453100,-9999.0,1.602679,1.795299,1.880034,71.729141,...,-9999.0,-0.597914,-9999.0,-9999.0,-9999.0,-9999.0,1.0,6,-1,0
5,495.288,437641.96875,6470972.5,-470.261780,19.453100,-9999.0,1.585567,1.804719,1.879687,72.014420,...,-9999.0,-0.601600,-9999.0,-9999.0,-9999.0,-9999.0,1.0,6,-1,0
6,495.440,437641.96875,6470972.5,-470.413788,19.462496,-9999.0,1.576569,1.805498,1.878731,72.588089,...,-9999.0,-0.598369,-9999.0,-9999.0,-9999.0,-9999.0,1.0,6,-1,0
7,495.592,437641.96875,6470972.5,-470.565796,19.468800,-9999.0,1.587011,1.808367,1.867837,71.283051,...,-9999.0,-0.602039,-9999.0,-9999.0,-9999.0,-9999.0,1.0,6,-1,0
8,495.744,437641.96875,6470972.5,-470.717773,19.468800,-9999.0,1.613674,1.815813,1.847233,69.721436,...,-9999.0,-0.614364,-9999.0,-9999.0,-9999.0,-9999.0,1.0,6,-1,0
9,495.896,437641.96875,6470972.5,-470.869781,19.468800,-9999.0,1.634622,1.813916,1.836309,66.677727,...,-9999.0,-0.621813,-9999.0,-9999.0,-9999.0,-9999.0,1.0,6,-1,0


### Model Training

The choice of algorithm for this tutorial workflow is xgboost. Why? Performance on previously done validation was better, and also at a faster compute speed than catboost. Random forest is also a great algorithm to try. Let's implement our xgboost tree. This will be done in a 10 fold cross validation technique. This is done to get a better performance and a confident result that is not due to randomness. We will be using StratifiedKFold function from sklearn. Let's look at that.

In [81]:
def show_evaluation(pred, true):

  '''

  function to show model performance and evaluation
  args:
    pred: predicted value(a list)
    true: actual values (a list)

  prints the custom metric performance, accuracy and F1 score of predictions

  '''

  print(f'Default score: {score(true.values, pred)}')
  print(f'Accuracy is: {accuracy_score(true, pred)}')
  print(f'F1 is: {f1_score(pred, true.values, average="weighted")}')

In [83]:
# initializing the xgboost model

model = xgb.XGBClassifier(n_estimators=100, max_depth=10, booster='gbtree',
                          objective='softprob', learning_rate=0.1, random_state=0,
                          subsample=0.9, colsample_bytree=0.9, tree_method='gpu_hist',
                          eval_metric='mlogloss', reg_lambda=1500, n_jobs=-1)

In [84]:
split = 10

kf = StratifiedKFold(n_splits=split, shuffle=True)

In [85]:
# initializing the prediction probabilities arrays

test_pred = np.zeros((len(test), 12))
valid1_pred = np.zeros((len(valid1), 12))
valid2_pred = np.zeros((len(valid2), 12))
valid_pred = np.zeros((len(valid), 12))

In [86]:
test_pred

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [53]:
#implementing the CV Loop

i = 1
for (train_index, test_index) in kf.split(train, lithology):
    print(train_index.shape, test_index.shape)
    print(train_index, test_index)
    X_train,X_test = train.iloc[train_index], train.iloc[test_index]
    Y_train,Y_test = lithology.iloc[train_index], lithology.iloc[test_index]
    
        
    model.fit(X_train, Y_train, early_stopping_rounds=100, eval_set=[(X_test, Y_test)], verbose=1)
    
    prediction1 = model.predict(valid1)
    prediction2 = model.predict(valid2)
    prediction = model.predict(valid)
    
    print(show_evaluation(prediction1, valid1_lithology))
    print(show_evaluation(prediction2, valid2_lithology))
    print(show_evaluation(prediction, valid_lithology))
 
    print(f'----------------------- FOLD {i} ---------------------')
    i+=1
    
    valid1_pred += model.predict_proba(valid1)
    valid2_pred += model.predict_proba(valid2)
    valid_pred += model.predict_proba(valid)

(847170,) (94131,)
[     0      1      2 ... 941298 941299 941300] [     3     12     17 ... 941254 941261 941297]


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[0]	validation_0-mlogloss:2.16357
Default score: -0.9455169215021432
Accuracy is: 0.6839187085492504
F1 is: 0.6969638976491602
None
Default score: -0.8728242084693497
Accuracy is: 0.6792874812212671
F1 is: 0.7068894813860023
None
Default score: -0.9174397932027398
Accuracy is: 0.6821299245233629
F1 is: 0.7001680772519585
None
----------------------- FOLD 1 ---------------------
(847171,) (94130,)
[     0      1      2 ... 941298 941299 941300] [     6     20     51 ... 941242 941258 941293]
[0]	validation_0-mlogloss:2.16257
Default score: -0.9691825005864415
Accuracy is: 0.6774856232984312
F1 is: 0.6894236403360214
None
Default score: -0.8684528583208142
Accuracy is: 0.6769041352746495
F1 is: 0.7084191911073783
None
Default score: -0.9302762750316304
Accuracy is: 0.6772610270058025
F1 is: 0.696218032206216
None
----------------------- FOLD 2 ---------------------
(847171,) (94130,)
[     0      1      2 ... 941297 941298 941300] [     4     13     18 ... 941290 941294 941299]
[0]	valid

In [45]:
# finding the probabilities average and converting the numpy array to a dataframe

valid1_pred = pd.DataFrame(valid1_pred/split)
valid2_pred = pd.DataFrame(valid2_pred/split)
valid_pred = pd.DataFrame(valid_pred/split)

In [46]:
# extracting the index position with the highest probability as the lithology classp

valid1_pred = valid1_pred.idxmax(axis=1)
valid2_pred = valid2_pred.idxmax(axis=1)
valid_pred = valid_pred.idxmax(axis=1)

Evaluating the final predictions from the CV and max probability indexing

In [47]:
show_evaluation(valid1_pred, valid1_lithology)

Default score: -0.9571631160301111
Accuracy is: 0.6801939166471186
F1 is: 0.6918461589943973


In [48]:
show_evaluation(valid2_pred, valid2_lithology)

Default score: -0.8554701178118399
Accuracy is: 0.6841784233771221
F1 is: 0.7175537752878195


In [49]:
show_evaluation(valid_pred, valid_lithology)

Default score: -0.9178847999650975
Accuracy is: 0.6817329086863575
F1 is: 0.7019928305401797
